Example from:

Wilson, Zachary T., and Nikolaos V. Sahinidis. "Automated learning of chemical reaction networks." Computers & Chemical Engineering 127 (2019): 88-98.
https://doi.org/10.1016/j.compchemeng.2019.05.020

*Case 1: Isothermal CSTR*

For isothermal CSTRs across a known range of feed concentrations, $C_s^l \leq C_s^0 \leq C_s^u$, s $\in$ F. 

The simulated reaction networks id defined below, where $k_1^{true} = 1.5$, $k_2^{true} = 2.1$, and $k_3^{true} = 0.9$ with a residence time of $\tau = 1$ is used for the reactor. 


$A + B \rightarrow C \quad  \{{k_1^{true}}\}$ 

$B + C \rightarrow D \quad  \{{k_2^{true}}\}$

$A + D \rightarrow E \quad  \{{k_3^{true}}\}$

Initial concentrations are specificed for species $F = {A,B}$ over the range $0 \leq C_s^0 \leq 10$, $s\in F$.

In [ ]:
# Imports

import pyomo.environ as pyo
from idaes.surrogate import ripe
import numpy as np
import random
import isotsim

np.random.seed(20)

# Setup the problem
noise = 0.1
ns = 5  # number of species
lb_conc = [0,0,0,0,0]
ub_conc = [10,10,0,0,0]

# initial concentrations - only 2 data points
cdata0 = [[1,1,0,0,0],[10,10,0,0,0]]
cdata = isotsim.sim(cdata0)
nd = len(cdata0) # number of data points

# Expected variance based off the noise in the data
sigma = np.multiply(noise**2,np.array(cdata))

The postulated set reaction stoichiometries is defined as:

$ A + B \rightarrow C $

$ B + C \rightarrow D $

$ A + D \rightarrow E $

$ A + 2B \rightarrow D $

$ 2A + 2B \rightarrow E $

$ A + B + C \rightarrow E $

$ 2A + B + D \rightarrow C + E$

$ C + D \rightarrow E + A $

$ 3A + 3B \rightarrow C + E $

$ 3A + 4B \rightarrow D + E $

$ 2A + 3B  \rightarrow C + D $

$ 4A + 5B \rightarrow C + D + E $

In [ ]:
# considered reaction stoichiometries
#            A   B   C   D   E
stoich = [[ -1, -1,  1,  0,  0],
          [  0, -1, -1,  1,  0],
          [ -1,  0,  0, -1,  1],
          [ -1, -2,  0,  1,  0],
          [ -2, -2,  0,  0,  1],
          [ -1, -1, -1,  0,  1],
          [ -2, -1,  1, -1,  1],
          [  1,  0, -1, -1,  1],
          [ -3, -3,  1,  0,  1],
          [ -3, -4,  0,  1,  1],
          [ -2, -3,  1,  1,  0],
          [ -4, -5,  1,  1,  1]]

We have the initial conditions and possible stoichiometries to consider, but we still need the kinetics reaction mechanisms. Reaction mechanisms require a stoichiometry and kinetic model. In this case, we will be using mass action kinetics for all the stoichiometries available, which is built into RIPE.

In [ ]:
# IRIPE internal mass action kinetics are specified
rxn_mechs = [['all','massact']]

Now we are ready to run the RIPE model builder:

In [ ]:
results = ripe.ripemodel(cdata,
                             stoich = stoich,
                             mechanisms=rxn_mechs,
                             x0=cdata0,
                             hide_output=False,
                             sigma=sigma,
                             deltaterm=0,
                             expand_output=True)

Based on the number of data points, the best model chosen is only one reaction.

$ 4A + 5B \rightarrow C + D + E $

So similar to how ALAMO iterates between developing a model and adding additional points with error maximization, RIPE provides methods to do the same.

In [ ]:
# Adaptive experimental design using error maximization sampling
[new_points, err] = ripe.ems(results,
                             isotsim.sim,
                             lb_conc,
                             ub_conc,
                             5, #number of species
                             x=cdata,
                             x0=cdata0) 
print("New Point", new_points)
print("Error", err)

# Implement EMS as described in the RIPE publication
new_res = isotsim.sim(new_points)[0]
print("New Result",new_res)

Running ripe.ems gives us additional points maximizing the error that we can use to develop a new model until our error tolerance is achieved. A common loop in using RIPE follows:

In [ ]:
ite = 0

while any(err >  [2*noise*s for s in new_res] ):
    print('Which concentrations violate error (True=violation) : ', err > [noise*s for s in new_res])
    results = {}
    ite+=1
    
    # Data updated explicitly 
    # so RBFopt subroutines produce consistent results
    
    new_cdata0 = np.zeros([nd+ite,ns])
    new_cdata  = np.zeros([nd+ite,ns])
    new_cdata0[:-1][:] = cdata0[:][:]
    new_cdata[:-1][:] = cdata[:][:]
    new_cdata0[-1][:] = new_points[:]
    res = isotsim.sim(new_points)[0]
    for j in range(len(res)):
        new_cdata[-1][j] = res[j]

    #Update weight parameters
    sigma =  np.multiply(noise**2,np.array(new_cdata))

    # Build updated RIPE model
    results = ripe.ripemodel(new_cdata, 
                             stoich = stoich,
                             mechanisms=rxn_mechs,
                             x0=new_cdata0,
                             sigma=sigma,
                             expand_output=True)

    # Another call to EMS
    [new_points, err] = ripe.ems(results,
                                 isotsim.sim,
                                 lb_conc,
                                 ub_conc,
                                 5,
                                 x=cdata,
                                 x0=cdata0)

    # Update results
    new_res = isotsim.sim(new_points)[0]
    cdata0 = new_cdata0
    cdata = new_cdata

The results can vary, but RIPE can identify the simulated system of:

$A + B \rightarrow C \quad  \{{k_1^{true}}\}$ 

$B + C \rightarrow D \quad  \{{k_2^{true}}\}$

$A + D \rightarrow E \quad  \{{k_3^{true}}\}$

In [ ]:
# Final call to RIPE to get concise output
results = ripe.ripemodel(cdata,
                         stoich = stoich,
                         mechanisms=rxn_mechs,
                         x0=cdata0,
                         sigma=sigma,
                         expand_output=False)
print(results)